# cheerio / axios를 이용해서 웹크롤링을 해보자~
SYNOPSIS<br>
    [.도움 (.help)]<br>
    [.뉴스 (.news)]<br>
<br>
DESCRIPTION<br>
    .help: 도움말<br>
    .news: 최신보안뉴스<br>
<br>
EXTENSION<br>
    [.실검 (.realtime)]<br>
    [.날씨 지역 (.weather local)]<br>
    [.환율 (.exchange)]<br>
    [.운세 (.luck)]<br>
    [.운세 띠 (.luck zodiac_name)]<br>
    [.로또 (.lotto)]<br>
    [.음식메뉴 (.foodmenu)]<br>
    [.소라고동 내용 (.conch text)]<br>
    [.노래추천 (.music)]<br>


## 네이버 뉴스 취득
* 이런식으로 패키지를 불러드릴 수 있다.
* 아래의 라이브러리가 안 깔려 있을 경우에 [npm install axios]와 같이 명령어를 실행해준다.
* 처음 시도할 시에는 [npm init] 명령어로 프로젝트를 작성할 필요가 있음..ㅎㅎ

In [57]:
var axios    = require('axios')
var cheerio  = require('cheerio')
var iconv    = require('iconv-lite')
var log      = console.log

### 아래의 URL의 사이트에서 20대에게 인기있는 10가지 기사를 취득해보자
* URL : https://news.naver.com/main/ranking/popularLike.nhn?subType=20



In [58]:
async function getNews() {
    var url   = 'https://news.naver.com/main/ranking/popularLike.nhn?subType=20'
    var NAVER = "https://news.naver.com"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url)
        data = data.data
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    $bodyList.each(function() {
    str += `
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
${NAVER + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
<br>
`
    })
    console.log(str)
}
getNews()


�� �������� �� ��ݹ��� �� ���� ��Ÿ���١�
<br>
[����Ź�] ��(������ ���� ���ϰ��������繫�� ���Ŀ�) ���ε��� �� ��ݹ��� �� ���� ��Ÿ�����ϴ�. (�迩�� �뵿�� ��1�κ��� ��ȭ��) ������ ���鼭��
<br>
https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=081&aid=0003099958&date=&type=1&rankingSectionId=001&rankingSeq=1
<br>
<br>

����� �ҸӴϿ� �Ŵ� 350���� ������, ������ ������
<br>
���Ǳ�￬��(���ǿ�)�� ��α�ǰ ȸ�� ���� ��Ȥ ���� ���� ���� ������ �Ϻ��� ���Ⱥ� �������� ����� �ҸӴ��� ���� Ȳ�𾾿� ����� ���𾾸� �ҡ�
<br>
https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=025&aid=0003010049&date=&type=0&rankingSectionId=001&rankingSeq=2
<br>
<br>

'�ڷγ� �б�' ���л� 1�δ� 20������ ��ϱ� ȯ�ҡ��ᱹ ��������?(����)
<br>
��ϱ� �Ϻ� ȯ�� ��å ���� �� ���� '1900��' 3�� �߰� �ݿ� ���� 1�δ� 20������ ���л� 195���� ȯ�� '�ڷγ� �б�'�� �Ҹ��� �̹� �б� ��ϱ� ȯ��
<br>
https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=366&aid=0000540821&date=&type=1&rankingSectionId=001&rankingSeq=3
<br>
<br>

"��

##### 아니.. 성공했다고 생각했더니 위와 같은 문자깨짐이 일어난다..;

아마도 문제는
euc-kr -> UTF-8로 변환을 하고 있기 때문일 것으로 추측..]
axios로 데이터를 취득 시 변환작업을 안하도록 설정을 해본다.



##### 이번에 접근할 방법은 iconv-lite로 [arrayBuffer]에서 [EUC-KR]로 취득하는 방법이 되겠다 ㅎㅎ

적용방법 : 
1. var iconv    = require('iconv-lite')
2. axios의 [responseType -> arraybuffer]
3. iconv.decode("인코딩을 변경할 문자열", "EUC-KR").toString()

In [59]:
async function getNews() {
    var url   = 'https://news.naver.com/main/ranking/popularLike.nhn?subType=20'
    var NAVER = "https://news.naver.com"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url, {method: "GET", responseType: "arraybuffer" })
        data = data.data
        data = iconv.decode(data, "EUC-KR").toString()
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    $bodyList.each(function() {
    str += `
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
${NAVER + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
<br>
`
    })
    console.log(str)
}
getNews()


文 “국민이 더 충격받은 것 같아 안타깝다”
<br>
[서울신문] “(북측의 개성 남북공동연락사무소 폭파에) 국민들이 더 충격받은 것 같아 안타깝습니다. (김여정 노동당 제1부부장 담화를) 국민이 보면서…
<br>
https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=081&aid=0003099958&date=&type=1&rankingSectionId=001&rankingSeq=1
<br>
<br>

길원옥 할머니에 매달 350만원 지원금, 가족은 몰랐다
<br>
정의기억연대(정의연)의 기부금품 회계 누락 의혹 등을 수사 중인 검찰이 일본군 위안부 피해자인 길원옥 할머니의 양자 황모씨와 며느리 조모씨를 소…
<br>
https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=025&aid=0003010049&date=&type=0&rankingSectionId=001&rankingSeq=2
<br>
<br>

'코로나 학기' 대학생 1인당 20만원씩 등록금 환불…결국 세금으로?(종합)
<br>
등록금 일부 환불 대책 나올 듯 당정 '1900억' 3차 추경 반영 검토 1인당 20만원씩 대학생 195만명 환불 '코로나 학기'로 불리는 이번 학기 등록금 환…
<br>
https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=366&aid=0000540821&date=&type=1&rankingSectionId=001&rankingSeq=3
<br>
<br>

"집도 없으면서" TBS 라디오에서 서민 비하한 김어준
<br>
국민세금으로 운영되는 TBS서 부적절 발언 통합당 "김어준의 편파공장" 방송인 김어준이 자신이 진행하는 라디오 방송 ‘김어준의 뉴스공장’에서 집 …
<br>
https://news.naver.com/main/ranking